In [ ]:
## CBAM활용

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 1) CBAM 모듈 정의
##############################
class CBAMBlock(nn.Module):
    """
    Convolutional Block Attention Module (CBAM)
    - Sanghyun Woo et al., "CBAM: Convolutional Block Attention Module", ECCV 2018
    """
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAMBlock, self).__init__()
        
        # --------------------------
        # 채널 어텐션 (Channel Attention)
        # --------------------------
        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False)
        )
        self.channel_sigmoid = nn.Sigmoid()
        
        # --------------------------
        # 공간 어텐션 (Spatial Attention)
        # --------------------------
        self.spatial_conv = nn.Conv2d(2, 1, kernel_size=kernel_size, 
                                      padding=kernel_size // 2, bias=False)
        self.spatial_sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x.shape: (B, C, H, W)
        
        # 1) Channel Attention
        ch_att = self.channel_att(x)               # (B, C, 1, 1)
        ch_att = self.channel_sigmoid(ch_att)      # (B, C, 1, 1)
        x = x * ch_att                             # 채널별로 스케일링
        
        # 2) Spatial Attention
        avg_out = torch.mean(x, dim=1, keepdim=True)         # (B, 1, H, W)
        max_out, _ = torch.max(x, dim=1, keepdim=True)       # (B, 1, H, W)
        x_sa = torch.cat([avg_out, max_out], dim=1)          # (B, 2, H, W)
        x_sa = self.spatial_conv(x_sa)                       # (B, 1, H, W)
        x_sa = self.spatial_sigmoid(x_sa)                    # (B, 1, H, W)
        x = x * x_sa                                         # 공간별로 스케일링
        
        return x


##############################
# 2) CustomEfficientNet 정의
##############################
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        
        # 1. 사전학습된 EfficientNet 불러오기
        self.backbone = EfficientNet.from_pretrained(model_name)
        
        # 2. (옵션) 일부 레이어 동결
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        # 3. 기존 FC 레이어 제거(Identity)
        in_features = self.backbone._fc.in_features  # b0의 경우 1280 채널
        self.backbone._fc = nn.Identity()
        
        # 4. 추가 CNN 블록 (CBAM 적용 예시)
        self.extra_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_features, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            CBAMBlock(channels=512, reduction=16, kernel_size=7),  # CBAM 추가
            nn.AdaptiveAvgPool2d((1, 1))                           # (B, 512, 1, 1)
        )
        
        # 5. 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # (a) EfficientNet Backbone 특징 추출 (extract_features)
        x = self.backbone.extract_features(x)  # (B, in_features, H, W)
        
        # (b) 추가 블록 (CBAM 포함)
        x = self.extra_conv(x)                 # (B, 512, 1, 1)
        
        # (c) 분류기 통과
        x = x.view(x.size(0), -1)             # (B, 512)
        x = self.classifier(x)                # (B, num_classes)
        
        return x
    
# 하이퍼파라미터
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 & 로더
image_size = EfficientNet.get_image_size(model_name)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 및 손실 함수, 최적화기
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 및 평가 함수
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# 학습 루프
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0


NameError: name 'EarlyStopping' is not defined